In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Load a data
housing = pd.read_csv(filepath_or_buffer="dataset/housing.csv",)

#Sampling a data
housing["income_cat"] = pd.cut(housing["median_income"], bins=[0,1.5,3,4.5,6,np.inf],labels = [1,2,3,4,5])
split = StratifiedShuffleSplit(n_splits =1 , test_size = 0.2, random_state =42)
for train_index,test_index in split.split(X= housing, y=housing["income_cat"]):
    strat_train_set = housing.iloc[train_index]
    strat_test_set = housing.iloc[test_index]
strat_test_set.drop(columns=["income_cat"],inplace = True)
strat_train_set.drop(columns=["income_cat"],inplace = True)

# define a predictors and target
housing = strat_train_set.drop("median_house_value",axis=1)
housing_labels = strat_train_set[["median_house_value"]]

C:\Users\GK\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
#Transformation Pipelines

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self # nothing else to do
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

transformer_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
 ])

num_attribs = list(housing)
num_attribs.remove('ocean_proximity')
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([("transformer_pipeline", transformer_pipeline, num_attribs),("OneHotEncoder", OneHotEncoder(), cat_attribs)])
housing_prepared = full_pipeline.fit_transform(housing)


In [17]:
import pickle
with open('processing_pipeline.pkl','wb') as f:
    pickle.dump(full_pipeline,file=f)

In [23]:
with open('processing_pipeline.pkl','rb') as file:
    pipleline = pickle.load(file=file)

EOFError: Ran out of input